In [0]:
! wget https://raw.githubusercontent.com/hse-python-2nd-2019/seminars/master/UDPipe/model.py

--2019-12-18 06:53:38--  https://raw.githubusercontent.com/hse-python-2nd-2019/seminars/master/UDPipe/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2067 (2.0K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   2.02K  --.-KB/s    in 0s      

2019-12-18 06:53:39 (39.4 MB/s) - ‘model.py’ saved [2067/2067]



In [0]:
! pip install ufal.udpipe

     |████████████████████████████████| 307kB 2.8MB/s 
  Created wheel for ufal.udpipe: filename=ufal.udpipe-1.2.0.3-cp36-cp36m-linux_x86_64.whl size=5625838 sha256=01e2ff81f12b38a5de7ee8be5ea311f0e4e9eb70b892c1556683aa9e1de567a3
  Stored in directory: /root/.cache/pip/wheels/0c/9d/db/6d3404c33da5b7adb6c6972853efb6a27649d3ba15f7e9bebb
Successfully built ufal.udpipe


In [0]:
PATH = './'

langs = [
    ('de', 'german-gsd-ud-2.4-190531.udpipe'),
    ('sv', 'swedish-talbanken-ud-2.4-190531.udpipe'),
    ('da', 'danish-ddt-ud-2.4-190531.udpipe'),
    ('en', 'english-gum-ud-2.4-190531.udpipe')
]

In [0]:
import ufal.udpipe
from model import Model
from tqdm.auto import tqdm
import os
import re

In [0]:
import sqlite3
db = sqlite3.connect('./translations.db')
cur = db.cursor()

In [0]:
for lang in ['en', 'da', 'sv', 'de']:
    cur.execute(f'ALTER TABLE yandex_{lang} ADD udpipe TEXT')
    cur.execute(f'ALTER TABLE google_{lang} ADD udpipe TEXT')
    db.commit()

In [0]:
lang = 'en'
source = 'yandex'
cur.execute(f'SELECT id, text FROM {source}_{lang} WHERE udpipe IS NULL LIMIT 3')
texts = cur.fetchall()
texts

[(1, 'Ufeig was a man named .'),
 (2, 'He lived in the north the story starts in the Dungeon .'),
 (3,
  'He was Skíðason but his mother was called Gunnlaug , the daughter of Ófeigs from Skörðum .')]

In [0]:
def get_conllu(pair, model):
    idx, text = pair
    text = text.replace('.', '. ')
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu = model.write(sentences, "conllu")
    conllu = re.sub('# newdoc\n# newpar\n# sent_id = 1\n', '', conllu)
    return conllu, idx

In [0]:
data = []
model = Model(os.path.join(PATH, langs[3][1]))
for pair in tqdm(texts):
    data.append(get_conllu(pair))

In [0]:
cur.executemany(f'UPDATE {source}_{lang} SET udpipe = ? WHERE id = ?', data)
db.commit()

In [0]:
langs = {
    'de': 'german-gsd-ud-2.4-190531.udpipe',
    'sv': 'swedish-talbanken-ud-2.4-190531.udpipe',
    'da': 'danish-ddt-ud-2.4-190531.udpipe',
    'en': 'english-gum-ud-2.4-190531.udpipe'
}

In [0]:
cur.execute(f'SELECT count(id) FROM {source}_{lang} WHERE udpipe IS NULL')
cur.fetchone()

(101246,)

In [0]:
def parse_batch(source, lang, step=100):
    cur.execute(f'SELECT id, text FROM {source}_{lang} WHERE udpipe IS NULL LIMIT {step}')
    texts = cur.fetchall()
    data = []
    model = Model(os.path.join(PATH, langs[lang]))
    for pair in tqdm(texts, leave=False):
        try:
          data.append(get_conllu(pair, model))
        except Exception as e:
          print(e)
    cur.executemany(f'UPDATE {source}_{lang} SET udpipe = ? WHERE id = ?', data)
    db.commit()
    if len(data) > 0:
        return True
    else:
        return False

In [0]:
cur.execute("UPDATE yandex_da SET udpipe = NULL")
db.commit()

In [0]:
source = 'google'
step = 1000
for lang in ['en', 'da', 'sv', 'de']:
    print(lang)
    cur.execute(f'SELECT count(id) FROM {source}_{lang} WHERE udpipe IS NULL')
    n = int(cur.fetchone()[0] / step + 1)
    for _ in tqdm(range(n)):
        parse_batch(source=source, lang=lang, step=step)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! ls -sh /content/translations.db

682M /content/translations.db


In [0]:
! cp "/content/translations.db" "/content/drive/My Drive/Исследования/DH 19-20 Формульные конструкции/Данные"